# ================

# PROJETO SIGALEI

# ================

#### REQUIREMENTS:
fake-useragent
lxml
numpy
pandas
request
urllib3
itertools
hashlib

In [1]:
import hashlib
import requests
import pandas as pd
from lxml import html
from datetime import datetime, timedelta, date
from fake_useragent import UserAgent
import json
from math import ceil
import os
import itertools

In [2]:
ano_atual = datetime.today().year
hj = date.today()
dias_3 = hj - timedelta(days=3)

# FUNÇÕES

***

## * retorna_props()
<style>
    /* Jupyter */
    .rendered_html table,
    /* Jupyter Lab*/
    div[data-mime-type="text-markdown"] table {
        margin-left: 0
    }
</style>

Recebe a resposta em JSON da API da Camara e extrai dados **data de apresentação, titulo da proposição e id da proposição**.o

Ex.:

data de apresentação | titulo da proposição | id da proposição
-- | -- | --
2021-09-17T15:25:00 | PL 3211/2021 | 2299232

A *Id da proposição* é usada para completar o link onde está localizado o pdf de __*Inteiro Teor*__ e fazer o download.
Cada pd é salvo temporariamente e desse arquivo temporario é extraído o hash md5.

Com isso, o md5 de cada proposicao é formando listas com **data de apresentação, titulo da proposição, id da proposição e md5**.

Ex.:

data de apresentação | titulo da proposição | id da proposição | md5
-- | -- | -- | --
2021-09-17 | PL 3211-2021 | 2299232 | 4d9eef77f177f4e9b4113f9163b575e5

A função retorna uma listas com essas listas.

In [3]:
def retorna_props(qtd, resp_json, dias_3):
    lista_preps = []
    i = 0
    while i < qtd:
        data = resp_json['hits']['hits'][i]['_source']['dataApresentacao'][:10]
        data = datetime.strptime(data, '%Y-%m-%d').date()
        titulo = resp_json['hits']['hits'][i]['_source']['titulo'].replace('/','-')
        id_preposicao = resp_json['hits']['hits'][i]['_id']
        i += 1
        if data >= dias_3:
            url_prep = f'https://www.camara.leg.br/proposicoesWeb/fichadetramitacao?idProposicao={id_preposicao}'
            resp = requests.get(url=url_prep)
            tree = html.fromstring(html=resp.text)
            link_pdf = tree.xpath('//*[@id="content"]/h3[1]/span[2]/a/@href')[0]
            elo = link_pdf[link_pdf.find('codteor'):]
            url_pdf = f'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?{elo}.pdf'
            chunk_size = 2000
            r = requests.get(url_pdf, stream=True)
            salva_pdf = 'salva_pdf'
            with open(f'{salva_pdf}.pdf', 'wb') as fd:
                for chunk in r.iter_content(chunk_size):
                    fd.write(chunk)
            path = f'{salva_pdf}.pdf'
            with open(path, 'rb') as opened_file:
                content = opened_file.read()
                md5 = hashlib.md5()
                md5.update(content)
                md5_pdf = md5.hexdigest()          
            lista_preps.append([data.strftime('%Y-%m-%d'), titulo, id_preposicao, md5_pdf])
        else:
            pass
    
    return lista_preps

## * acessa_api_camara()

Faz um __POST__ com os parametros da API (__*order, ano, pagina, tiposDeProposicao*__) no site da Camara e recebe um __JSON de resposta__.

Passa como parametros :

* o ano_atual → como há recesso parlamentar no final do ano, nao ha riscos de perder propostas nos 3 primeiros dias do ano.
* o tipo de proposicao → *PL, PLP ou PEC*.
* o numero da pagina → a api exporta cum conjunto de dados limitados por pagina, com isso, passamos o número de cada página.

[__API__   ](https://www.camara.leg.br/api/v1/busca/proposicoes/_search) USADA NO SITE DA CÂMARA

In [4]:
def acessa_api_camara(ano_atual, tipo_proposicao, numero_da_pagina):
    ua = UserAgent()
    url = "https://www.camara.leg.br/api/v1/busca/proposicoes/_search"
    payload = {
        "order":"data",
        "ano":ano_atual,
        "pagina":numero_da_pagina,
        "tiposDeProposicao": tipo_proposicao
    }
    headers = {
        'Content-Type': "application/json",
        'User-Agent': ua.random    
        }
    response = requests.post(url, data=json.dumps(payload), headers=headers)
    return response.json()

## * limita_props_dos_3_ultimos_dias()

Chama a função __acessa_api_camara()__ e recebe o __json com os dados das proposições__.

Busca a quantidade total de paginas com proposições e aloca na variavel *qtd_pags*.

Busca a última data da primeira página. Caso seja mais do que os 3 dias, retorna uam lista da __funcao retorna_props()__

Caso seja menos do que 2 dias, vai para a página seguinte e faz a mesma busca até que a data seja maior que 3 dias,
fazerndo um break e retornando a lista da função __retorna_props()__

In [5]:
def limita_props_dos_3_ultimos_dias(tipo_proposicao):    
    n_i = 1
    resp_json = acessa_api_camara(ano_atual, tipo_proposicao, n_i)
    qtd_total = resp_json['aggregations']['ano']['buckets'][0]['doc_count']
    qtd = len(resp_json['hits']['hits'])
    qtd_pags = ceil(qtd_total / qtd)
    datau = resp_json['hits']['hits'][(qtd-1)]['_source']['dataApresentacao'][:10]
    datau = datetime.strptime(datau, '%Y-%m-%d').date()
    if datau >= dias_3:
        lista_preposicoes = []
        while n_i < qtd_pags:        
            resp_json = acessa_api_camara(ano_atual, tipo_proposicao, n_i)
            n_i += 1
            qtd_paginado = len(resp_json['hits']['hits'])
            rr = retorna_props(qtd_paginado, resp_json, dias_3)
            if rr == []:
                break        
            lista_preposicoes.append(rr)
        return list(itertools.chain.from_iterable(lista_preposicoes))
    else:
        return retorna_props(qtd, resp_json, dias_3)

## * exporta_dataframe()

Exporta um dataframe para o tipo de proposição *PL, PLP ou PEC*.

Remove o pdf usado para fazer o hash md5 da pasta.

Monta e retorna um dataframe com a lista recebida.

In [6]:
def exporta_dataframe(tipo_proposicao):
    try:
        salva_pdf = 'salva_pdf'
        path = f'{salva_pdf}.pdf'        
        os.remove(path)
    except:
        pass
    try:
        sigalei = limita_props_dos_3_ultimos_dias(tipo_proposicao)        
        df = pd.DataFrame(sigalei, columns = ['DATA', 'PROJETO', 'INDEX_PROJETO', 'MD5'])
    except:    
        sigalei = limita_props_dos_3_ultimos_dias(tipo_proposicao)
        loop = len(sigalei)
        print(len)
        x = 0
        df_list = []
        while x < loop:
            df_loop = pd.DataFrame(sigalei[x], columns = ['DATA', 'PROJETO', 'INDEX_PROJETO', 'MD5'])
            df_list.append(df_loop)
            x +=1
        df = pd.concat(df_list)
    return df    

## * salva_csv()

Salva o dataframe em CSV

In [7]:
def salva_csv(df, p):
    return df.to_csv(f'{p}.csv', index=False, sep=';')

## * salva_json()

Salva o dataframe em JSON

In [8]:
def salva_json(df, p):    
    retorno_json = df.to_json(orient='records')
    with open(f'{p}.json', 'w') as fout:
        json.dump(json.loads(retorno_json) , fout)

***
***
***

# RODANDO 
### ESCOLHA UMA DAS PROPOSIÇÕES: *PL, PLC ou PEC*

*Pode demorar algo em torno de 30 segundos*

In [9]:
# 'PL', 'PLP', 'PEC'

tipo_proposicao=input().upper()

plp


***

Gerando o Dataframe da proposição

In [10]:
df_sigalei = exporta_dataframe(tipo_proposicao)

***

Explorando o Dataframe

In [11]:
df_sigalei.head()

,DATA,PROJETO,INDEX_PROJETO,MD5
0,2021-09-16,PLP 140-2021,2299129,246dcbbe054bc801ab2a70e644506ff4
1,2021-09-16,PLP 139-2021,2299128,d1763f86a3fbfd50f91e923279685b36


In [12]:
df_sigalei.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   DATA           2 non-null      object
 1   PROJETO        2 non-null      object
 2   INDEX_PROJETO  2 non-null      object
 3   MD5            2 non-null      object
dtypes: object(4)
memory usage: 96.0+ bytes


***

Exportando para arquibo __CSV__

In [13]:
salva_csv(df_sigalei, tipo_proposicao)

***

Exportando para arquibo __JSON__

In [14]:
salva_json(df_sigalei, tipo_proposicao)